In [9]:
# Fourier Option Pricing: Option pricing using characteristic function 
# Import modules 
import math
import numpy as np
from scipy.stats import norm
import scipy.integrate as integrate
import matplotlib.pyplot as plt

In [10]:
# Market, option,share dynamics and information
r     = 0.06 # continously compounded risk free rate
S_0   = 100  # initial stock price
sigma = 0.3  # constant volatility of the share price
K     = 110  # Strike price of call option
T     = 1    # Time until maturity 
k_log = np.log(K)

In [41]:
# Pricing functions, Fourier characteristic function aspectes, and Assumptions
# - All of the Black Scholes Model pricing assumptions
# Call option price -> S_0*Prob_Q_S[S_T>K] - K*exp(-r*T)*Prob_Q[S_T>K]

# - Characteristic function of risk neutral probability density of share_price / ln(share_price) is applied
# - Gil-Pelaez Theorem(used to calculate probabilities given characteristic function) is applied 
# - Gil-pelaez Prob[X<=x] = 1/2 - 1/pi(Integral[-inf,inf]Im(exp(itx*char(x)/t dt)))
# - Im - is imaginery part complex number, char - characteristic function;  1j is imaginery number in Python

# Characteristic function where M1 = ln(S) under risk neutral measure Q
def car_M1(t):
    return np.exp(1j*t*(np.log(S_0) + (r-0.5*sigma**2)*T )-0.5*(sigma**2)*T*t**2)

# Characteristic function of price where M2= ln(S) under risk netural measure Q_S
def car_M2(t):
    return  np.exp(1j*t*sigma**2*T)*car_M1(t)

# Estimating integral using areas of rectangles for sufficient upper bound t_max divided into N parts
# Each small part is delta_t = t_max/N
# Each t_k used in estimate is (k-1/2)*delta_t for repective k=1,2,3,4...N

# Initialisin and calculating parameters for estimation t_max, N, t_delta
t_max    = 20
N        = 100
delta_t  = t_max/N
# The array of various t_k points
from_1_N = np.linspace(1,N,N)
t_k      = (from_1_N-1/2)*delta_t

# To use actual value not estimate can use intergrate function intergrate(lambda x: f(x), lower_bound,upper_bound)

# Approximate intergral estimates for 
first_intergral   = sum((((np.exp(-1j*t_k*k_log))*car_M2(t_k)).imag/t_k)*delta_t)
second_intergral  = sum((((np.exp(-1j*t_k*k_log))*car_M1(t_k)).imag/t_k)*delta_t)

In [42]:
# Calculating closed-form solution for call price
d_1 = (np.log(S_0/K) + (r+sigma**2/2)*T)/sigma*np.sqrt(T)
d_2 = d_1 - sigma*np.sqrt(T)
call_price_analytic = S_0*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)
call_price_analytic

10.424100458714285

In [44]:
# Calculating estimate call price using characteric functions using estimated integral_with_M
call_price_fourier = S_0*(1/2 + first_intergral/np.pi) - np.exp(-r*T)*K*(1/2 + second_intergral/np.pi)
call_price_fourier

10.424100443080057